NHỮNG BƯỚC CHÂN ĐẦU BAO GIỜ CŨNG KHÓ KHĂN NHẤT, NGẠI ĐỌC ĐỀ BÀI. 

# 1. GẶM 1 MIẾNG, 1 2 3 4 5, Cứ chạy thử đã HOÀN THÀNH TRƯỚC, HOÀN HẢO XONG, CHIẾN!

### **Giải thích đơn giản về bài toán OTTO Recommender System**

Hãy tưởng tượng bạn đang **mua sắm trực tuyến** trên một trang web lớn như Shopee hay Lazada. Khi bạn đang tìm kiếm sản phẩm, có rất nhiều lựa chọn, nhưng có thể bạn không biết chọn cái nào. Để giúp bạn, trang web sử dụng một **hệ thống gợi ý sản phẩm** nhằm đưa ra những sản phẩm mà bạn có thể **quan tâm** hoặc **muốn mua** dựa trên những gì bạn đã làm trước đó.

#### **Vậy bài toán này cần làm gì?**

Mục tiêu của bạn là **dự đoán những gì người dùng sẽ làm tiếp theo** trong một **phiên mua sắm (session)** dựa trên các hành động mà họ đã thực hiện trước đó.

Cụ thể, bạn sẽ cần dự đoán:

1. **Người dùng sẽ nhấp vào sản phẩm nào (clicks)?**
2. **Họ sẽ thêm sản phẩm nào vào giỏ hàng (carts)?**
3. **Họ sẽ đặt mua sản phẩm nào (orders)?**

### **Dữ liệu là gì?**

Dữ liệu bạn có là các phiên mua sắm trước đó của người dùng, với các thông tin:

- **session_id**: Phiên làm việc của một người dùng.
- **timestamp (ts)**: Thời điểm người dùng thực hiện hành động.
- **product_id (aid)**: Mã sản phẩm mà người dùng tương tác.
- **event_type**: Loại hành động, có thể là:
  - `clicks`: Nhấp vào xem sản phẩm.
  - `carts`: Thêm sản phẩm vào giỏ hàng.
  - `orders`: Đặt mua sản phẩm.

Ví dụ đơn giản:

| session_id | ts         | product_id | event_type |
|------------|------------|------------|------------|
| 123        | 12:00:01  | 101        | clicks     |
| 123        | 12:01:20  | 102        | carts      |
| 123        | 12:03:10  | 105        | orders     |

- Trong phiên **123**, người dùng đã nhấp vào sản phẩm **101**, sau đó thêm sản phẩm **102** vào giỏ hàng và cuối cùng đặt mua sản phẩm **105**.

### **Bài toán yêu cầu bạn làm gì?**

Dựa trên những hành động đã diễn ra trong **một phần của phiên làm việc**, bạn cần **dự đoán**:

- Sản phẩm nào người dùng có thể **nhấp vào**, **thêm vào giỏ hàng**, hoặc **đặt mua** **tiếp theo**.

### **Tóm tắt dễ hiểu**

Hãy tưởng tượng bạn là một **nhân viên bán hàng** trong một cửa hàng. Dựa vào việc **quan sát** khách hàng đang xem gì, cầm sản phẩm nào lên và cho vào giỏ, bạn sẽ đoán xem họ có thể **mua** sản phẩm nào tiếp theo. Đây chính là điều mà hệ thống gợi ý cần làm trong bài toán này — **dự đoán nhu cầu của người dùng** dựa trên lịch sử hành vi của họ.

Và bạn chỉ có một nhiệm vụ: **Đưa ra danh sách tối đa 20 sản phẩm mà bạn nghĩ khách hàng sẽ quan tâm** để giúp họ tìm thấy những gì họ muốn mua nhanh hơn!

## 1.2 Cách làm đơn giản nhất - frequency_based_recommender - top_popular_items_recommender - most_frequent_items


Để dự đoán một cách đơn giản nhất, chúng ta có thể sử dụng một phương pháp cơ bản dựa trên **tần suất xuất hiện của các sản phẩm** trong dữ liệu huấn luyện. Phương pháp này không cần sử dụng đến các mô hình phức tạp, mà chỉ cần thống kê xem **những sản phẩm nào được nhấp chuột, thêm vào giỏ hàng, và đặt mua nhiều nhất**.

### **Chiến lược cơ bản nhất**
1. **Đếm số lần xuất hiện của từng sản phẩm** cho mỗi loại hành động (`clicks`, `carts`, `orders`) trong dữ liệu huấn luyện.
2. **Sắp xếp sản phẩm theo thứ tự từ nhiều đến ít** cho từng loại hành động.
3. Khi dự đoán cho dữ liệu test, chỉ cần **chọn ra top 20 sản phẩm phổ biến nhất** cho mỗi loại hành động (`clicks`, `carts`, `orders`).

### **Cách thực hiện (Pseudocode)**
1. Đọc dữ liệu huấn luyện.
2. Tính tần suất xuất hiện của từng sản phẩm cho từng loại hành động.
3. Tạo danh sách top 20 sản phẩm phổ biến nhất cho mỗi loại hành động (`clicks`, `carts`, `orders`).
4. Khi dự đoán cho mỗi session trong tập dữ liệu test:
   - Sử dụng danh sách top 20 sản phẩm phổ biến đã tạo trước đó để dự đoán.

---

### **Triển khai bằng Python**
Dưới đây là mã Python để thực hiện phương pháp đơn giản nhất này.

```python
import pandas as pd
from collections import Counter

# Bước 1: Đọc dữ liệu huấn luyện
train_df = pd.read_csv('train.csv')

# Bước 2: Đếm tần suất sản phẩm cho từng loại hành động
clicks_count = Counter(train_df[train_df['event_type'] == 'clicks']['aid'])
carts_count = Counter(train_df[train_df['event_type'] == 'carts']['aid'])
orders_count = Counter(train_df[train_df['event_type'] == 'orders']['aid'])

# Bước 3: Lấy top 20 sản phẩm phổ biến nhất cho mỗi loại hành động
top_20_clicks = [item[0] for item in clicks_count.most_common(20)]
top_20_carts = [item[0] for item in carts_count.most_common(20)]
top_20_orders = [item[0] for item in orders_count.most_common(20)]

# Bước 4: Đọc dữ liệu test
test_df = pd.read_csv('test.csv')

# Bước 5: Dự đoán cho mỗi session trong tập test
predictions = []

for session_id in test_df['session_id'].unique():
    # Dự đoán cho từng loại hành động
    predictions.append(f"{session_id}_clicks," + " ".join(map(str, top_20_clicks)))
    predictions.append(f"{session_id}_carts," + " ".join(map(str, top_20_carts)))
    predictions.append(f"{session_id}_orders," + " ".join(map(str, top_20_orders)))

# Bước 6: Ghi kết quả ra file submission.csv
with open('submission.csv', 'w') as f:
    f.write("session_type,labels\n")
    for line in predictions:
        f.write(line + "\n")

print("Dự đoán hoàn tất! File submission.csv đã được tạo.")
```

---

### **Giải thích mã Python**
1. **Đọc dữ liệu huấn luyện** từ file `train.csv`.
2. Sử dụng **Counter** để đếm số lần xuất hiện của mỗi sản phẩm (`aid`) cho từng loại hành động (`clicks`, `carts`, `orders`).
3. **Lấy top 20 sản phẩm** phổ biến nhất cho từng loại hành động.
4. Đọc dữ liệu test và dự đoán dựa trên danh sách top 20 sản phẩm đã tạo.
5. Ghi kết quả dự đoán ra file `submission.csv`.

---

### **Ưu và nhược điểm của phương pháp này**
- **Ưu điểm**:
  - **Rất đơn giản và nhanh chóng** để triển khai.
  - Không yêu cầu sử dụng các mô hình phức tạp.
- **Nhược điểm**:
  - **Thiếu tính cá nhân hóa**: Mọi người dùng đều nhận được cùng một danh sách sản phẩm gợi ý.
  - Không tối ưu khi có nhiều **dữ liệu phức tạp và thay đổi nhanh chóng**.

Phương pháp này chỉ phù hợp để có một dự đoán nhanh và đơn giản. Để cải thiện độ chính xác, cần phải sử dụng các phương pháp phức tạp hơn như **mô hình học máy, deep learning** hoặc **các mô hình gợi ý dựa trên lịch sử** của từng người dùng.

## 1.3 Quay lại bài toán gốc 
1. Cần thiết làm bài toán không ? 
2. Input, Output và cách thức đo lường, Vấn đề bài toán giải quyết (khoảng cách từ A đến B): <Xác định điểm A, điểm B, ,khoảng cách Gap A với B>.
3. Demo, testing, Beta

### **1. Mục tiêu bài toán**

#### **Mục tiêu**
Xây dựng **hệ thống gợi ý sản phẩm** cho một trang thương mại điện tử lớn (như OTTO). Hệ thống cần dự đoán **các hành động tiếp theo** của người dùng trong phiên mua sắm, bao gồm:

1. **Click vào sản phẩm**.
2. **Thêm sản phẩm vào giỏ hàng**.
3. **Đặt mua sản phẩm**.

Mục tiêu của hệ thống này là giúp **cải thiện trải nghiệm người dùng** và **tăng doanh số bán hàng** cho trang web.

---

#### **Input và Output**

**Input:**
- **Dữ liệu huấn luyện (train.jsonl)**: Bao gồm các phiên làm việc (sessions) của người dùng, mỗi phiên có:
  - `session_id`: Mã định danh phiên làm việc.
  - `ts (timestamp)`: Thời gian xảy ra hành động.
  - `aid (article_id)`: Mã sản phẩm mà người dùng tương tác.
  - `event_type`: Loại hành động, bao gồm:
    - `clicks`: Nhấp vào sản phẩm.
    - `carts`: Thêm sản phẩm vào giỏ hàng.
    - `orders`: Đặt mua sản phẩm.

**Ví dụ một phiên làm việc:**
| session_id | ts        | aid     | event_type |
|------------|-----------|---------|------------|
| 12345678   | 165234560 | 1002938 | clicks     |
| 12345678   | 165234590 | 1002942 | carts      |
| 12345678   | 165234620 | 1002977 | orders     |


Input
```json 
{
  "session": 123456,
  "events": [
    {
      "aid": 1517085,
      "ts": 1659304800025,
      "type": "clicks"
    },
    {
      "aid": 1563459,
      "ts": 1659304904511,
      "type": "carts"
    },
    {
      "aid": 1602938,
      "ts": 1659304950123,
      "type": "orders"
    }
  ]
}

```
Ví dụ về 1 phiên session trong data: 
- events: 
+, aid: Mã sản phẩm mà người dùng tương tác.
+, ts: Thời gian xảy ra hành động.
+, type: Loại hành động (clicks, carts, orders).

- **Dữ liệu kiểm tra (test.jsonl)**: Gồm các phiên bị cắt ngắn, chỉ chứa một phần lịch sử phiên làm việc của người dùng. Nhiệm vụ của bạn là **dự đoán các hành động tiếp theo** (clicks, carts, orders) sau thời điểm cuối cùng trong mỗi phiên.

**Output:**
- **File submission.csv**: Dự đoán các sản phẩm mà người dùng có thể click, thêm vào giỏ hàng, hoặc đặt mua tiếp theo cho mỗi session.
  - Định dạng:
    ```
    session_type,labels
    12906577_clicks,135193 129431 119318
    12906577_carts,135193 129431 119318
    12906577_orders,135193 129431 119318
    12906578_clicks,135193 129431 119318
    ```
  - Mỗi dòng gồm:
    - **session_type**: Mã phiên làm việc kèm theo loại hành động (`clicks`, `carts`, `orders`).
    - **labels**: Tối đa 20 sản phẩm mà bạn dự đoán, cách nhau bởi dấu cách.


Dưới đây là **giải thích chi tiết** về **file submission** mà bạn cần nộp cho bài toán **OTTO Multi-Objective Recommender System**.

### **Mục tiêu của file submission**
File `submission.csv` chứa **kết quả dự đoán** của bạn về các sản phẩm mà người dùng sẽ tương tác trong tương lai, dựa trên dữ liệu từ `test.jsonl`. Mục tiêu là dự đoán **top 20 sản phẩm** mà người dùng có thể:
1. **Nhấp chuột vào (clicks)**
2. **Thêm vào giỏ hàng (carts)**
3. **Đặt mua (orders)**

---

### **Cấu trúc của file `submission.csv`**

File `submission.csv` cần có **2 cột**:

| session_type      | labels                      |
|-------------------|-----------------------------|
| 12899779_clicks   | 129004 126836 118524 ...    |
| 12899779_carts    | 129004 126836 118524 ...    |
| 12899779_orders   | 129004 126836 118524 ...    |
| 12899780_clicks   | 129004 126836 118524 ...    |
| 12899780_carts    | 129004 126836 118524 ...    |
| 12899780_orders   | 129004 126836 118524 ...    |

#### **Giải thích các cột:**
1. **`session_type`**:
   - Định danh phiên (`session_id`) kết hợp với **loại hành động** (`clicks`, `carts`, hoặc `orders`).
   - Ví dụ: `12899779_clicks` nghĩa là **dự đoán sản phẩm người dùng sẽ click** trong phiên `12899779`.

2. **`labels`**:
   - Danh sách **tối đa 20 sản phẩm (`aid`)** mà bạn dự đoán cho phiên và loại hành động tương ứng.
   - Các sản phẩm trong danh sách được cách nhau bởi **khoảng trắng**.
   - Ví dụ: `129004 126836 118524` nghĩa là bạn dự đoán người dùng sẽ tương tác với các sản phẩm có mã `129004`, `126836`, và `118524`.

---

### **Ví dụ chi tiết về file `submission.csv`**

Giả sử bạn có **2 phiên** trong file `test.jsonl`:

```json
{
  "session": 12899779,
  "events": [
    {"aid": 59625, "ts": 1661724000278, "type": "clicks"}
  ]
},
{
  "session": 12899780,
  "events": [
    {"aid": 126321, "ts": 1661724100289, "type": "clicks"}
  ]
}
```

**Dự đoán của bạn có thể như sau:**

```
session_type,labels
12899779_clicks,129004 126836 118524 134213 125678 128953 127654 129888 123456 124789 126222 128000 123321 122345 127111 128732 129567 120876 124567 126345
12899779_carts,129004 126836 118524 134213 125678 128953 127654 129888 123456 124789 126222 128000 123321 122345 127111 128732 129567 120876 124567 126345
12899779_orders,129004 126836 118524 134213 125678 128953 127654 129888 123456 124789 126222 128000 123321 122345 127111 128732 129567 120876 124567 126345
12899780_clicks,129004 126836 118524 134213 125678 128953 127654 129888 123456 124789 126222 128000 123321 122345 127111 128732 129567 120876 124567 126345
12899780_carts,129004 126836 118524 134213 125678 128953 127654 129888 123456 124789 126222 128000 123321 122345 127111 128732 129567 120876 124567 126345
12899780_orders,129004 126836 118524 134213 125678 128953 127654 129888 123456 124789 126222 128000 123321 122345 127111 128732 129567 120876 124567 126345
```

### **Lưu ý quan trọng khi tạo file `submission.csv`**
1. **Dòng đầu tiên** của file phải là **header**:
   ```
   session_type,labels
   ```
2. **Mỗi `session_id` cần có 3 loại hành động** (`clicks`, `carts`, `orders`).
3. **Tối đa 20 sản phẩm cho mỗi hàng**. Nếu bạn dự đoán ít hơn 20 sản phẩm, chỉ cần liệt kê số sản phẩm bạn có.
4. Các giá trị `aid` trong cột `labels` phải được **cách nhau bởi khoảng trắng**.
5. **Không để trống bất kỳ hàng nào** trong file.

---



---

### **2. Các metrics và cách thức đo lường**

Để đánh giá hiệu quả của hệ thống gợi ý, **Recall@20** được sử dụng. 

#### **Recall@20 là gì?**
- **Recall@20** đo lường khả năng của hệ thống gợi ý **dự đoán đúng sản phẩm** mà người dùng có thể click, thêm vào giỏ hàng, hoặc đặt mua.
- Cụ thể, nếu sản phẩm thực tế mà người dùng nhấp vào hoặc mua nằm trong **top 20 sản phẩm** mà hệ thống gợi ý, hệ thống sẽ được tính điểm.

#### **Công thức tính điểm tổng**
Điểm số cuối cùng được tính dựa trên trung bình có trọng số của Recall cho từng loại hành động:
\[
\text{score} = 0.10 \times R_{\text{clicks}} + 0.30 \times R_{\text{carts}} + 0.60 \times R_{\text{orders}}
\]

Trong đó:
- \( R_{\text{type}} \) là **Recall@20** cho từng loại hành động (`clicks`, `carts`, `orders`).
- \( N \) là tổng số phiên trong tập kiểm tra.
- Công thức tính Recall cho từng loại hành động:
  \[
  R_{\text{type}} = \frac{\sum_{i=1}^{N} | \{ \text{predicted aids} \}_{i,\text{type}} \cap \{ \text{ground truth aids} \}_{i,\text{type}} |}{\sum_{i=1}^{N} \min(20, | \{ \text{ground truth aids} \}_{i,\text{type}} |)}
  \]

---

### **Tóm tắt cách tính điểm:**
- **Recall@20** sẽ kiểm tra xem **các sản phẩm gợi ý** của bạn có chứa sản phẩm mà người dùng đã thực sự tương tác không.
- Đối với **clicks**, chỉ có **một sản phẩm** đúng cần dự đoán.
- Đối với **carts** và **orders**, có thể có **nhiều sản phẩm** đúng.

### **Kết luận**
- Để đạt điểm cao, bạn cần **tối ưu hóa** dự đoán cho cả 3 loại hành động với trọng số khác nhau (`clicks`, `carts`, `orders`).
- Đối với bài toán này, tập trung vào việc tối ưu dự đoán cho **orders** sẽ giúp bạn đạt điểm cao hơn, vì trọng số của nó là **60%**.

### **Gợi ý chiến lược đơn giản ban đầu:**
- Sử dụng **phương pháp đếm tần suất** để dự đoán các sản phẩm phổ biến nhất.
- Cải tiến dần bằng cách sử dụng các mô hình học máy hoặc deep learning để cá nhân hóa dự đoán cho từng phiên.

## 1.4 Copy and Development: : LÀM LUÔN, KO CẦN ĐỢI HOÀN HẢO, HỌC ĐƯỢC 1/3 DÙNG LUÔN, KHÔNG CẦN ĐỢI HỌC HẾT MỚI LÀM. 

Reference: https://youtu.be/e-I_G9QhHTA?feature=shared - Project 07: Hybrid Recommendation System Using Machine Learning
(code kèm theo: https://github.com/Chando0185/Multiverse_of_100-_data_science_project_series/tree/main/Hybrid%20Recommendation%20System%20using%20Python)

### **Tóm tắt về Hybrid Recommender System**

| Phương pháp                  | **Content-Based Filtering**                                      | **Collaborative Filtering**                                    | **Hybrid Recommender System**                                         |
|------------------------------|------------------------------------------------------------------|----------------------------------------------------------------|-----------------------------------------------------------------------|
| **Cách hoạt động**          | - Dựa trên **nội dung** của mục (phim, nhạc, sách).              | - Dựa trên **hành vi người dùng** như **lượt nhấp, mẫu hành vi, xếp hạng**. | - Kết hợp cả **content-based** và **collaborative filtering** để tận dụng ưu điểm của cả hai. |
|                             | - Sử dụng thuộc tính như **thể loại, tác giả, diễn viên** để gợi ý các mục tương tự. | - Dự đoán dựa trên **những người dùng có hành vi tương tự** hoặc **các mục tương tự** mà người dùng đã tương tác. | - Có nhiều cách kết hợp như **Weighted Hybrid**, **Switching Hybrid**, **Feature Augmentation**. |
| **Ví dụ**                   | - Gợi ý phim hành động nếu người dùng vừa xem một bộ phim hành động. | - Nếu người dùng A và B có lịch sử xem tương tự, gợi ý cho B phim mà A thích. | - **Netflix** gợi ý phim dựa trên cả **thể loại yêu thích** và **hành vi của người dùng khác**. |
| **Ưu điểm**                 | - Không cần dữ liệu từ người dùng khác.                           | - Tìm ra các mục **mới và đa dạng** dựa trên cộng đồng người dùng. | - **Giảm thiểu cold start** và **cải thiện độ chính xác** trong gợi ý. |
|                             | - Tập trung vào **sở thích cá nhân** của từng người dùng.        | - Tận dụng **hành vi của những người dùng tương tự** để đưa ra gợi ý. | - Kết hợp hai phương pháp giúp **tối ưu hóa** trải nghiệm người dùng. |
| **Nhược điểm**              | - Hạn chế trong việc gợi ý các mục mới (**exploration problem**). | - Cần **nhiều dữ liệu** từ người dùng để hoạt động hiệu quả (**cold start problem**). | - **Phức tạp hơn** khi triển khai và cần tối ưu hóa trọng số hoặc điều kiện chuyển đổi. |
|                             | - Phụ thuộc vào **thuộc tính** của các mục và có thể bỏ sót các mục tiềm năng. | - Khó gợi ý các mục mới nếu chưa có ai tương tác với chúng. | - Cần **xử lý và tối ưu** cả hai nguồn dữ liệu (nội dung và hành vi). |

---

### **Các kỹ thuật Hybrid phổ biến**
| **Kỹ thuật Hybrid**           | **Mô tả**                                                                                                      |
|------------------------------|----------------------------------------------------------------------------------------------------------------|
| **Weighted Hybrid**         | Kết hợp điểm số từ cả hai phương pháp với các **trọng số khác nhau**.                                            |
| **Switching Hybrid**        | Sử dụng một phương pháp tại một thời điểm, tùy thuộc vào **ngữ cảnh hoặc dữ liệu có sẵn**.                         |
| **Feature Augmentation**    | Dùng kết quả của một phương pháp để **bổ sung đặc trưng** cho phương pháp còn lại.                                |

---

### **Tóm tắt**
Phương pháp **hybrid recommender system** kết hợp giữa **content-based** và **collaborative filtering** giúp cải thiện hiệu suất gợi ý bằng cách tận dụng các đặc điểm của cả hai. Đây là phương pháp được sử dụng rộng rãi trong các hệ thống như **Netflix, YouTube, Spotify** để cung cấp trải nghiệm tốt hơn cho người dùng. 🚀